<a href="https://colab.research.google.com/github/kyaoNK/bert_nli/blob/main/CREATE_DATASET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
!sudo apt -y update
# !sudo apt -y -q install libeigen3-dev libprotobuf-dev protobuf-c-compiler

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://ppa.launchpad.ne

In [ ]:
!pip install -qU pytorch-lightning lightning-bolts 
!pip install -qU torch
!pip install -qU torchtext
!pip install -qU torchvision
!pip install -qU torchaudio
!pip install -q   datasets
!pip install -q   accelerate
!pip install -q   ipadic
!pip install -q   unidic-lite
!pip install -qU transformers
!pip install -q   sentencepiece
# !pip install -qU sentence_transformers
# !pip install -q   pyknp
# !pip install -qU torch==1.7.1
# !pip install -qU torchtext==0.8.0
# !pip install -qU torchvision==0.8.2
# !pip install -qU torchaudio==0.7.2
# !pip install -q   transformers==4.4.2
# !pip install -q   pytorch_ligtning==1.2.2

     |████████████████████████████████| 800 kB 18.7 MB/s 
     |████████████████████████████████| 329 kB 51.7 MB/s 
     |████████████████████████████████| 125 kB 64.8 MB/s 
     |████████████████████████████████| 512 kB 68.5 MB/s 
     |██████████████████████████████  | 834.1 MB 1.2 MB/s eta 0:00:45tcmalloc: large alloc 1147494400 bytes == 0x395f2000 @  0x7f4d8d23b615 0x5d6f4c 0x51edd1 0x51ef5b 0x4f750a 0x4997a2 0x4fd8b5 0x4997c7 0x4fd8b5 0x49abe4 0x4f5fe9 0x55e146 0x4f5fe9 0x55e146 0x4f5fe9 0x55e146 0x5d8868 0x5da092 0x587116 0x5d8d8c 0x55dc1e 0x55cd91 0x5d8941 0x49abe4 0x55cd91 0x5d8941 0x4990ca 0x5d8868 0x4997a2 0x4fd8b5 0x49abe4
     |████████████████████████████████| 887.4 MB 1.8 kB/s 
     |████████████████████████████████| 557.1 MB 12 kB/s 
     |████████████████████████████████| 21.0 MB 1.2 MB/s 
     |████████████████████████████████| 849 kB 57.7 MB/s 
     |████████████████████████████████| 317.1 MB 31 kB/s 
ERROR: pip's dependency resolver does not currently take into accou

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/my-modules')

### Juman++

In [ ]:
# %cd /content/drive/MyDrive/Colab Notebooks/nlp-resource/JUMANPP/jumanpp-2.0.0-rc3
# !mkdir build
# %cd build
# !rm -f CMakeCache.txt
# !cmake .. -DCMAKE_BUILD_TYPE=Release -DCMAKE_INSTALL_PREFIX=/usr/local
# !make
# !sudo make install
# !cd /content

In [ ]:
# from pyknp import Juman
# jumanpp = Juman()   # default is JUMAN++: Juman(jumanpp=True). if you use JUMAN, use Juman(jumanpp=False)
# result = jumanpp.analysis("下鴨神社の参道は暗かった。")
# for mrph in result.mrph_list(): # 各形態素にアクセス
#     print("見出し:%s, 読み:%s, 原形:%s, 品詞:%s, 品詞細分類:%s, 活用型:%s, 活用形:%s, 意味情報:%s, 代表表記:%s" \
#             % (mrph.midasi, mrph.yomi, mrph.genkei, mrph.hinsi, mrph.bunrui, mrph.katuyou1, mrph.katuyou2, mrph.imis, mrph.repname))

# Datasets

In [ ]:
import csv

def tsv2csv (tsv_filepath) :
    with open(tsv_filepath, 'r') as tsv_f:
        tsv = csv.reader(tsv_f, delimiter = '\t')
        csv_filepath = tsv_filepath[:-4] + '.csv'
        with open(csv_filepath, 'w', newline='\n') as csv_f:
            writer = csv.writer(csv_f, delimiter=",")
            for row in tsv:
                writer.writerow(row)
            return csv_filepath

In [ ]:
#@title JGLUE
import pandas as pd

JGLUE_DIRPATH = '/content/drive/MyDrive/Colab Notebooks/SeniorThesis/DATASETS/JGLUE/datasets/jnli-v1.1/'
JGLUE_FILENAME = { 'train' : 'train-v1.1.json' , 'valid' : 'valid-v1.1.json' }
JGLUE_COLS = [ 'sentence1' , 'sentence2' , 'label' ]
COLUMNS = [ 'sentenceA' , 'sentenceB' , 'label' ]

jglue_train_df = pd.read_json( JGLUE_DIRPATH + JGLUE_FILENAME['train'] , lines=True , encoding='utf-8' )
jglue_train_df = jglue_train_df.loc[ : , JGLUE_COLS ]
jglue_train_df.columns = COLUMNS
jglue_valid_df = pd.read_json( JGLUE_DIRPATH + JGLUE_FILENAME['valid'] , lines=True , encoding='utf-8' )
jglue_valid_df = jglue_valid_df.loc[ : , JGLUE_COLS ]
jglue_valid_df.columns = COLUMNS

# jglue_train_df

In [ ]:
#@title JSICK
import pandas as pd

JSICK_DIRPATH = '/content/drive/MyDrive/Colab Notebooks/SeniorThesis/DATASETS/JSICK/jsick/'
JSICK_FILENAME = { 'train' : 'train.tsv' , 'valid' : 'test.tsv' }
JSICK_COLS = [ 'sentence_A_Ja' , 'sentence_B_Ja' , 'entailment_label_Ja' ]
COLUMNS = [ 'sentenceA' , 'sentenceB' , 'label' ]

jsick_train_filepath = tsv2csv(JSICK_DIRPATH + JSICK_FILENAME['train'])
jsick_valid_filepath = tsv2csv(JSICK_DIRPATH + JSICK_FILENAME['valid'])

jsick_train_df = pd.read_csv( jsick_train_filepath , encoding='utf-8' , header=0 )
jsick_train_df = jsick_train_df.loc[ : , JSICK_COLS ]
jsick_train_df.columns = COLUMNS
jsick_valid_df = pd.read_csv( jsick_valid_filepath , encoding='utf-8' , header=0 )
jsick_valid_df = jsick_valid_df.loc[ : , JSICK_COLS ]
jsick_valid_df.columns = COLUMNS

jsick_train_df.index = len(jglue_train_df.index) + jsick_train_df.index
jsick_valid_df.index = len(jglue_valid_df.index) + jsick_valid_df.index

# jsick_train_df